# Instalação



In [ ]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]
!pip install stable_baselines3
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
#  Display Virtual para gravar vídeos
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

# Importações

In [14]:
# Ambientes
import gym
# Recursos RL
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
# from stable_baselines3.common.logger import configure
# Recursos de Video
from gym.wrappers import RecordVideo

# Treinamento

## Criação do Ambiente

`make_vec_env `  permite treinar um agente em vários ambientes em paralelo. Como a maioria dos algoritmos de aprendizado por reforço são muito famintos por "experiência", ter vários ambientes rodando em paralelo faz com que o agente aprenda rapidamente como se comportar.

In [15]:
ambiente = make_vec_env('LunarLander-v2', 4)

## Criação do Agente

Aqui você encontra a descrição dos Parâmetros da PPO:<br>
<https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html>

In [ ]:
# Instanciar o Agente
# agente = PPO("MlpPolicy", ambiente, verbose=1)
agente = PPO(
    policy = 'MlpPolicy',
    env = ambiente,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    verbose=1,
    tensorboard_log="LogtensorPPO")
# Treinar o Agente
agente.learn(total_timesteps=5000000, tb_log_name="PrimeiroTreino")
# Salvar o Agente
nomeAgente = "agenteLunarLander"
agente.save(nomeAgente)

# Avaliação do Agente

In [ ]:
ambienteAva = gym.make("LunarLander-v2")
media_recompensas, desvio_padrao_recompensas = evaluate_policy(agente, ambienteAva, n_eval_episodes=10, deterministic=True)
print(f"Média de Recompensas: {media_recompensas:.2f} Desvio Padrão de Recompensas: {desvio_padrao_recompensas:.2f}")


## Gravar um vídeo do Agente Treinado

In [6]:
ambienteAva = gym.make("LunarLander-v2")
ambienteAva = RecordVideo(ambienteAva, './video' )

In [ ]:
# Descomente essa linha se quiser gravar um vídeo de um modelo salvo
# agente = PPO.load("/content/agenteLunarLander.zip", ambienteAva)


In [11]:
obs = ambienteAva.reset()
concluido = False
while not concluido:
  acao, _estado = agente.predict(obs)
  obs, recompensa, concluido, info = ambienteAva.step(acao)
